In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from collections import deque
import random
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
%matplotlib notebook

from sklearn.model_selection import KFold ,train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
import networkx as nx

import imageio
from celluloid import Camera
from IPython.display import HTML
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'

In [2]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

                
#Shuffle the data because the raw data is ordered based on the label
n = len(all_data)
e = len(all_edges)
print("node =",len(all_data))
print('edge = ',len(all_edges))
random_state = 77
all_data = shuffle(all_data,random_state=random_state)
labels = []
nodes = []
X = []

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(elements[-1])
    X.append(elements[1:-1])
    nodes.append(elements[0])

X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)
data_dir = os.path.expanduser("cora")
edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["target", "source"])
edgelist["label"] = "cites"
edgelist.to_csv('edgelist.csv')
Gnx = nx.from_pandas_edgelist(edgelist, edge_attr="label")
nx.set_node_attributes(Gnx, "paper", "label")
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)
node_data.to_csv('node_data.csv')
a = pd.read_csv('edgelist.csv')
G = nx.from_pandas_edgelist(a, create_using=nx.DiGraph())


node = 2708
edge =  5429
X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {'Case_Based', 'Neural_Networks', 'Probabilistic_Methods', 'Reinforcement_Learning', 'Theory', 'Rule_Learning', 'Genetic_Algorithms'}

Number of classes:  7


In [3]:
l=[]
for i in range(0,len(edge_list)):
    u=int(edge_list[i][0]) 
    v=int(edge_list[i][1])
    if(l.count(u)==0):
        l.append(u)
    if(l.count(v)==0):
        l.append(v)
k=random.randint(0, len(l))
val=l[k]
print(val)


9581


In [4]:
neighbours=[]
for i in range(0,len(edge_list)):
    u=int(edge_list[i][0])
    v=int(edge_list[i][1])
    if(u==val):
        if(neighbours.count(v)==0):
            neighbours.append(v)
    if(v==val):
        if(neighbours.count(u)==0):
            neighbours.append(u)
print(neighbours)

[1130780, 633585, 31489]


In [10]:
import sys,os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import stellargraph as sg
from sklearn.model_selection import KFold ,train_test_split 
from sklearn.preprocessing import LabelBinarizer 
import tensorflow as tf
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from tensorflow.keras import layers, optimizers, losses, metrics, Model

data_dir = os.path.expanduser("cora")
column_names =  ["subject"]
nodes= pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)
nodes=pd.DataFrame(nodes).reset_index()
nodes=pd.DataFrame(np.array(nodes['subject']),index=nodes['level_0'],columns=['subject'])

def Gmodel(data_dir,K,lr,epochs):        
    data_dir = os.path.expanduser(data_dir)
    column_names =  ["subject"]
    data= pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)
    data=pd.DataFrame(data).reset_index()
    data=pd.DataFrame(np.array(data['subject']),index=data['level_0'],columns=['subject'])
    loader=sg.datasets.Cora()
    G,nodes=loader.load()
    generator = FullBatchNodeGenerator(G, method="gcn")
    fold=KFold(n_splits=K, shuffle=True, random_state=1) 
    encode=LabelBinarizer()
    prediction_=np.zeros((1,2708,7))
    for train_index,test_index in fold.split(data):
        x_train,x_test=data.iloc[train_index],data.iloc[test_index] 
        train_target=encode.fit_transform(x_train)       
        test_target=encode.fit_transform(x_test)
        train_gen = generator.flow(x_train.index, train_target)
        graph_conv = GCN(layer_sizes=[16, 16], activations=["relu", "relu"], generator=generator, dropout=0.5)                                                                                               
        x_input, x_output = graph_conv.in_out_tensors()  
        prediction = layers.Dense(units=train_target.shape[1], activation="softmax")(x_output)                                                                     
        model = Model(inputs=x_input, outputs=prediction)
        model.compile(optimizer=tf.optimizers.Adam(lr),loss=losses.categorical_crossentropy,metrics=["acc"],)
        test_gen = generator.flow(x_test.index, test_target)
        history = model.fit(train_gen,epochs=epochs,validation_data=test_gen,verbose=True,shuffle=False)
        entire_graph=data.index
        entire_generator = generator.flow(entire_graph)
        entire_predictions = model.predict(entire_generator)
        print(entire_predictions)
        print(entire_predictions.shape)
        print(type(entire_predictions))
        prediction_+= entire_predictions
        pred = prediction_/K      
        output_to_string = encode.inverse_transform(pred.squeeze())
        print(output_to_string)
        print(type(output_to_string))
    return pd.DataFrame({"paper_id": nodes.index, "class_label": output_to_string})

In [19]:
results=Gmodel("cora",K=10,lr=0.01,epochs=200)

Using GCN (local pooling) filters...
Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.9402 - acc: 0.1748 - val_loss: 1.8897 - val_acc: 0.2694
Epoch 2/200
1/1 [==============================] - 0s 181ms/step - loss: 1.8825 - acc: 0.3229 - val_loss: 1.8222 - val_acc: 0.2620
Epoch 3/200
1/1 [==============================] - 0s 175ms/step - loss: 1.7977 - acc: 0.3488 - val_loss: 1.7376 - val_acc: 0.2620
Epoch 4/200
1/1 [==============================] - 0s 169ms/step - loss: 1.7124 - acc: 0.3377 - val_loss: 1.6452 - val_acc: 0.2731
Epoch 5/200
1/1 [==============================] - 0s 178ms/step - loss: 1.6173 - acc: 0.3705 - val_loss: 1.5456 - val_acc: 0.3026
Epoch 6/200
1/1 [==============================] - 0s 174ms/step - loss: 1.5376 - acc: 0.3882 - val_loss: 1.4383 - val_acc: 0.4354
Epoch 7/200
1/1 [==============================] - 0s 180ms/step - loss: 1.4401 - acc: 0.4875 - val_loss: 1.3265 - val_acc: 0.5941
Epoch 8/200
1/1 [==============================] 

Epoch 63/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2755 - acc: 0.9060 - val_loss: 0.3678 - val_acc: 0.8745
Epoch 64/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2953 - acc: 0.9023 - val_loss: 0.3707 - val_acc: 0.8745
Epoch 65/200
1/1 [==============================] - 0s 168ms/step - loss: 0.2674 - acc: 0.9138 - val_loss: 0.3744 - val_acc: 0.8856
Epoch 66/200
1/1 [==============================] - 0s 183ms/step - loss: 0.2908 - acc: 0.9015 - val_loss: 0.3778 - val_acc: 0.8819
Epoch 67/200
1/1 [==============================] - 0s 183ms/step - loss: 0.2696 - acc: 0.9073 - val_loss: 0.3815 - val_acc: 0.8856
Epoch 68/200
1/1 [==============================] - 0s 182ms/step - loss: 0.2717 - acc: 0.9040 - val_loss: 0.3850 - val_acc: 0.8856
Epoch 69/200
1/1 [==============================] - 0s 172ms/step - loss: 0.2796 - acc: 0.9032 - val_loss: 0.3884 - val_acc: 0.8782
Epoch 70/200
1/1 [==============================] - 0s 166ms/step - loss: 0.

Epoch 125/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2164 - acc: 0.9216 - val_loss: 0.4360 - val_acc: 0.8782
Epoch 126/200
1/1 [==============================] - 0s 171ms/step - loss: 0.2195 - acc: 0.9188 - val_loss: 0.4339 - val_acc: 0.8782
Epoch 127/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2202 - acc: 0.9229 - val_loss: 0.4320 - val_acc: 0.8745
Epoch 128/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2134 - acc: 0.9241 - val_loss: 0.4317 - val_acc: 0.8782
Epoch 129/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2103 - acc: 0.9224 - val_loss: 0.4324 - val_acc: 0.8745
Epoch 130/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2315 - acc: 0.9134 - val_loss: 0.4351 - val_acc: 0.8782
Epoch 131/200
1/1 [==============================] - 0s 162ms/step - loss: 0.2023 - acc: 0.9294 - val_loss: 0.4382 - val_acc: 0.8745
Epoch 132/200
1/1 [==============================] - 0s 165ms/step - 

1/1 [==============================] - 0s 175ms/step - loss: 0.1887 - acc: 0.9307 - val_loss: 0.4794 - val_acc: 0.8745
Epoch 187/200
1/1 [==============================] - 0s 173ms/step - loss: 0.1884 - acc: 0.9343 - val_loss: 0.4786 - val_acc: 0.8745
Epoch 188/200
1/1 [==============================] - 0s 173ms/step - loss: 0.1862 - acc: 0.9294 - val_loss: 0.4779 - val_acc: 0.8745
Epoch 189/200
1/1 [==============================] - 0s 174ms/step - loss: 0.1849 - acc: 0.9282 - val_loss: 0.4770 - val_acc: 0.8782
Epoch 190/200
1/1 [==============================] - 0s 175ms/step - loss: 0.1821 - acc: 0.9319 - val_loss: 0.4799 - val_acc: 0.8782
Epoch 191/200
1/1 [==============================] - 0s 170ms/step - loss: 0.1872 - acc: 0.9348 - val_loss: 0.4846 - val_acc: 0.8745
Epoch 192/200
1/1 [==============================] - 0s 175ms/step - loss: 0.2042 - acc: 0.9229 - val_loss: 0.4893 - val_acc: 0.8708
Epoch 193/200
1/1 [==============================] - 0s 163ms/step - loss: 0.1940 -

1/1 [==============================] - 0s 181ms/step - loss: 0.3643 - acc: 0.8925 - val_loss: 0.3640 - val_acc: 0.9004
Epoch 43/200
1/1 [==============================] - 0s 161ms/step - loss: 0.4052 - acc: 0.8699 - val_loss: 0.3608 - val_acc: 0.8967
Epoch 44/200
1/1 [==============================] - 0s 183ms/step - loss: 0.3461 - acc: 0.8921 - val_loss: 0.3582 - val_acc: 0.8967
Epoch 45/200
1/1 [==============================] - 0s 167ms/step - loss: 0.3833 - acc: 0.8785 - val_loss: 0.3538 - val_acc: 0.8967
Epoch 46/200
1/1 [==============================] - 0s 169ms/step - loss: 0.3647 - acc: 0.8839 - val_loss: 0.3465 - val_acc: 0.8930
Epoch 47/200
1/1 [==============================] - 0s 199ms/step - loss: 0.3728 - acc: 0.8859 - val_loss: 0.3412 - val_acc: 0.8967
Epoch 48/200
1/1 [==============================] - 0s 162ms/step - loss: 0.3568 - acc: 0.8958 - val_loss: 0.3397 - val_acc: 0.9041
Epoch 49/200
1/1 [==============================] - 0s 169ms/step - loss: 0.3599 - acc: 0

...

Epoch 148/200
1/1 [==============================] - 0s 173ms/step - loss: 0.2090 - acc: 0.9253 - val_loss: 0.4285 - val_acc: 0.8630
Epoch 149/200
1/1 [==============================] - 0s 175ms/step - loss: 0.2094 - acc: 0.9249 - val_loss: 0.4265 - val_acc: 0.8667
Epoch 150/200
1/1 [==============================] - 0s 182ms/step - loss: 0.2234 - acc: 0.9258 - val_loss: 0.4253 - val_acc: 0.8667
Epoch 151/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2057 - acc: 0.9245 - val_loss: 0.4259 - val_acc: 0.8667
Epoch 152/200
1/1 [==============================] - 0s 179ms/step - loss: 0.2073 - acc: 0.9266 - val_loss: 0.4263 - val_acc: 0.8630
Epoch 153/200
1/1 [==============================] - 0s 169ms/step - loss: 0.2121 - acc: 0.9225 - val_loss: 0.4273 - val_acc: 0.8667
Epoch 154/200
1/1 [==============================] - 0s 180ms/step - loss: 0.2084 - acc: 0.9266 - val_loss: 0.4279 - val_acc: 0.8667
Epoch 155/200
1/1 [==============================] - 0s 175ms/step - 

Epoch 4/200
1/1 [==============================] - 0s 222ms/step - loss: 1.7356 - acc: 0.3318 - val_loss: 1.6636 - val_acc: 0.3222
Epoch 5/200
1/1 [==============================] - 0s 179ms/step - loss: 1.6709 - acc: 0.3519 - val_loss: 1.5719 - val_acc: 0.3593
Epoch 6/200
1/1 [==============================] - 0s 174ms/step - loss: 1.5779 - acc: 0.3749 - val_loss: 1.4751 - val_acc: 0.4037
Epoch 7/200
1/1 [==============================] - 0s 175ms/step - loss: 1.4940 - acc: 0.3950 - val_loss: 1.3824 - val_acc: 0.4556
Epoch 8/200
1/1 [==============================] - 0s 180ms/step - loss: 1.4432 - acc: 0.4212 - val_loss: 1.2948 - val_acc: 0.4963
Epoch 9/200
1/1 [==============================] - 0s 175ms/step - loss: 1.3483 - acc: 0.4643 - val_loss: 1.2104 - val_acc: 0.5444
Epoch 10/200
1/1 [==============================] - 0s 178ms/step - loss: 1.2761 - acc: 0.5045 - val_loss: 1.1271 - val_acc: 0.6148
Epoch 11/200
1/1 [==============================] - 0s 172ms/step - loss: 1.2269 -

1/1 [==============================] - 0s 177ms/step - loss: 0.2931 - acc: 0.9044 - val_loss: 0.3386 - val_acc: 0.8889
Epoch 67/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2988 - acc: 0.9048 - val_loss: 0.3394 - val_acc: 0.8926
Epoch 68/200
1/1 [==============================] - 0s 177ms/step - loss: 0.2804 - acc: 0.9098 - val_loss: 0.3405 - val_acc: 0.9000
Epoch 69/200
1/1 [==============================] - 0s 174ms/step - loss: 0.2967 - acc: 0.8987 - val_loss: 0.3420 - val_acc: 0.9074
Epoch 70/200
1/1 [==============================] - 0s 180ms/step - loss: 0.3000 - acc: 0.9011 - val_loss: 0.3423 - val_acc: 0.9111
Epoch 71/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2747 - acc: 0.9007 - val_loss: 0.3430 - val_acc: 0.9037
Epoch 72/200
1/1 [==============================] - 0s 175ms/step - loss: 0.2867 - acc: 0.8962 - val_loss: 0.3428 - val_acc: 0.9074
Epoch 73/200
1/1 [==============================] - 0s 177ms/step - loss: 0.2656 - acc: 0

Epoch 128/200
1/1 [==============================] - 0s 179ms/step - loss: 0.2413 - acc: 0.9094 - val_loss: 0.3811 - val_acc: 0.8963
Epoch 129/200
1/1 [==============================] - 0s 170ms/step - loss: 0.2095 - acc: 0.9217 - val_loss: 0.3825 - val_acc: 0.8889
Epoch 130/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2244 - acc: 0.9155 - val_loss: 0.3837 - val_acc: 0.8815
Epoch 131/200
1/1 [==============================] - 0s 175ms/step - loss: 0.2176 - acc: 0.9212 - val_loss: 0.3864 - val_acc: 0.8778
Epoch 132/200
1/1 [==============================] - 0s 176ms/step - loss: 0.2287 - acc: 0.9130 - val_loss: 0.3888 - val_acc: 0.8778
Epoch 133/200
1/1 [==============================] - 0s 177ms/step - loss: 0.2275 - acc: 0.9171 - val_loss: 0.3900 - val_acc: 0.8778
Epoch 134/200
1/1 [==============================] - 0s 179ms/step - loss: 0.2103 - acc: 0.9217 - val_loss: 0.3919 - val_acc: 0.8778
Epoch 135/200
1/1 [==============================] - 0s 172ms/step - 

1/1 [==============================] - 0s 191ms/step - loss: 0.2142 - acc: 0.9176 - val_loss: 0.4445 - val_acc: 0.8667
Epoch 190/200
1/1 [==============================] - 0s 188ms/step - loss: 0.2056 - acc: 0.9212 - val_loss: 0.4444 - val_acc: 0.8741
Epoch 191/200
1/1 [==============================] - 0s 183ms/step - loss: 0.1882 - acc: 0.9274 - val_loss: 0.4427 - val_acc: 0.8741
Epoch 192/200
1/1 [==============================] - 0s 182ms/step - loss: 0.1899 - acc: 0.9295 - val_loss: 0.4406 - val_acc: 0.8741
Epoch 193/200
1/1 [==============================] - 0s 180ms/step - loss: 0.2069 - acc: 0.9217 - val_loss: 0.4384 - val_acc: 0.8741
Epoch 194/200
1/1 [==============================] - 0s 174ms/step - loss: 0.1951 - acc: 0.9270 - val_loss: 0.4368 - val_acc: 0.8741
Epoch 195/200
1/1 [==============================] - 0s 179ms/step - loss: 0.1984 - acc: 0.9266 - val_loss: 0.4365 - val_acc: 0.8704
Epoch 196/200
1/1 [==============================] - 0s 175ms/step - loss: 0.1788 -

In [20]:
results.to_csv('Result.tsv', sep='\t', index=False)
from sklearn.metrics import accuracy_score
print('percentage of samples with their label classified correctly: {}%'.format(accuracy_score(nodes.subject.values,results.class_label.values)*100))

percentage of samples with their label classified correctly: 97.41506646971935%


In [15]:
G2 = nx.Graph()
for i in range(0,len(edge_list)):
    u=int(edge_list[i][0])
    v=int(edge_list[i][1])
    G2.add_edge(u, v)

In [16]:
import networkx as nx
centrality = nx.eigenvector_centrality(G2)
print(['%s %0.2f'%(node,centrality[node]) for node in centrality])

['35 0.65', '1033 0.05', '103482 0.05', '103515 0.07', '1050679 0.05', '1103960 0.06', '1103985 0.05', '1109199 0.05', '1112911 0.05', '1113438 0.05', '1113831 0.05', '1114331 0.06', '1117476 0.05', '1119505 0.05', '1119708 0.06', '1120431 0.05', '1123756 0.05', '1125386 0.05', '1127430 0.05', '1127913 0.06', '1128204 0.05', '1128227 0.05', '1128314 0.05', '1128453 0.06', '1128945 0.05', '1128959 0.05', '1128985 0.05', '1129018 0.05', '1129027 0.05', '1129573 0.05', '1129683 0.06', '1129778 0.06', '1130847 0.06', '1130856 0.05', '1131116 0.05', '1131360 0.05', '1131557 0.05', '1131752 0.05', '1133196 0.05', '1133338 0.05', '1136814 0.06', '1137466 0.05', '1152421 0.06', '1152508 0.05', '1153065 0.05', '1153280 0.06', '1153577 0.06', '1153853 0.05', '1153943 0.06', '1154176 0.05', '1154459 0.06', '116552 0.05', '12576 0.09', '128540 0.05', '132806 0.06', '135130 0.06', '141342 0.07', '141347 0.05', '148170 0.05', '15670 0.05', '1688 0.09', '175291 0.05', '178727 0.05', '18582 0.05', '19

In [17]:
ans=0
ans1=0
for node in centrality:
    if(neighbours.count(node)>0):
        if(ans<centrality[node]):
            ans=centrality[node]
            ans1=node
print(ans1)

31489
